# Automated ML

TODO: Import Dependencies. In the cell below, import all the dependencies that you will need to complete the project.

In [36]:
from azureml.core import Workspace, Experiment
from azureml.core.compute import ComputeTarget, AmlCompute
from azureml.core.compute_target import ComputeTargetException
from azureml.widgets import RunDetails
from azureml.train.hyperdrive.run import PrimaryMetricGoal
import os
import joblib
from azureml.core.dataset import Dataset
from azureml.train.automl import AutoMLConfig
from azureml.core.model import Model
from azureml.core.webservice import AciWebservice
from azureml.core.environment import Environment
from azureml.core.model import InferenceConfig
import requests
import json
import sklearn
from azureml.automl.runtime.onnx_convert import OnnxConverter
import pandas as pd
from pandas import DataFrame

## Dataset

### Overview
TODO: In this markdown cell, give an overview of the dataset you are using. Also mention the task you will be performing.


TODO: Get data. In the cell below, write code to access the data you will be using in this project. Remember that the dataset needs to be external.

In [37]:
ws = Workspace.from_config()

# choose a name for experiment
experiment_name = 'Heart-Disease-Predection'

experiment=Experiment(ws, experiment_name)

In [38]:
#Creating Compute
from azureml.core.compute import ComputeTarget, AmlCompute
from azureml.core.compute_target import ComputeTargetException
#Create compute cluster


# choose a name for your cluster
cluster_name= "cpu-cluster"

try:
    compute_target = ComputeTarget(workspace=ws , name=cluster_name)
    print('Found existing compute target')
except ComputeTargetException:
    print('Creating a new compute target ...')
    compute_config = AmlCompute.provisioning_configuration(vm_size='Standard_D2_V2' , max_nodes=4)
    #create the cluster
    cpu_cluster=ComputeTarget.create(ws,cluster_name,compute_config)
    print('cpu-cluster has been created')


Found existing compute target


In [39]:
#Try to load the dataset from the Workspace. Otherwise, create it from the file
# NOTE: update the key to match the dataset name
found = False
key = "Heart Failure Dataset"

if key in ws.datasets.keys(): 
        found = True
        dataset = ws.datasets[key] 
        print("Found")

if not found:
        # Create AML Dataset and register it into Workspace
        example_data = 'https://raw.githubusercontent.com/BMUHAMMDI/nd00333-capstone/master/starter_file/heart_failure_clinical_records_dataset.csv'
        dataset = Dataset.Tabular.from_delimited_files(example_data)        
        #Register Dataset in Workspace
        dataset = dataset.register(workspace=ws,
                                   name=key)


df = dataset.to_pandas_dataframe()
df.describe()

Found


,age,anaemia,creatinine_phosphokinase,diabetes,ejection_fraction,high_blood_pressure,platelets,serum_creatinine,serum_sodium,sex,smoking,time,DEATH_EVENT
count,299.00,299.00,299.00,299.00,299.00,299.00,299.00,299.00,299.00,299.00,299.00,299.00,299.00
mean,60.83,0.43,581.84,0.42,38.08,0.35,263358.03,1.39,136.63,0.65,0.32,130.26,0.32
std,11.89,0.50,970.29,0.49,11.83,0.48,97804.24,1.03,4.41,0.48,0.47,77.61,0.47
min,40.00,0.00,23.00,0.00,14.00,0.00,25100.00,0.50,113.00,0.00,0.00,4.00,0.00
25%,51.00,0.00,116.50,0.00,30.00,0.00,212500.00,0.90,134.00,0.00,0.00,73.00,0.00
50%,60.00,0.00,250.00,0.00,38.00,0.00,262000.00,1.10,137.00,1.00,0.00,115.00,0.00
75%,70.00,1.00,582.00,1.00,45.00,1.00,303500.00,1.40,140.00,1.00,1.00,203.00,1.00
max,95.00,1.00,7861.00,1.00,80.00,1.00,850000.00,9.40,148.00,1.00,1.00,285.00,1.00


In [40]:
from sklearn.model_selection import train_test_split

training_data, test_data = train_test_split(df, test_size=0.20)

## AutoML Configuration

TODO: Explain why you chose the automl settings and cofiguration you used below.

In [41]:
from azureml.data.dataset_factory import TabularDatasetFactory
if not os.path.isdir("data"):
    os.mkdir("data")

pd.DataFrame(training_data).to_csv("data/train_data.csv", index=False)
ds=ws.get_default_datastore()
ds.upload(src_dir='./data', target_path='HeartDataset', overwrite=True,show_progress=True)
train_data=TabularDatasetFactory.from_delimited_files(path=ds.path('HeartDataset/train_data.csv'))
print(train_data.to_pandas_dataframe())

Uploading an estimated of 2 files
Uploading ./data/test_ds.csv
Uploaded ./data/test_ds.csv, 1 files out of an estimated total of 2
Uploading ./data/train_data.csv
Uploaded ./data/train_data.csv, 2 files out of an estimated total of 2
Uploaded 2 files
      age  anaemia  creatinine_phosphokinase  diabetes  ejection_fraction  \
0   70.00        1                        69         1                 50   
1   65.00        0                       167         0                 30   
2   60.00        0                        59         0                 25   
3   66.00        1                        68         1                 38   
4   70.00        0                      2695         1                 40   
..    ...      ...                       ...       ...                ...   
234 55.00        0                       582         1                 35   
235 70.00        0                       835         0                 35   
236 64.00        1                        62         0  

In [45]:
# TODO: Put your automl settings here
automl_settings = {"experiment_timeout_minutes": 25,
                    "max_concurrent_iterations":4,                     
}

# TODO: Put your automl config here
automl_config = AutoMLConfig(  
    task='classification',
    compute_target='cpu-cluster',
    primary_metric = 'accuracy',
    training_data= train_data,
    label_column_name ='DEATH_EVENT',
    enable_onnx_compatible_models=True,
    featurization='auto',
    enable_early_stopping= True,
    **automl_settings
)

In [46]:
# TODO: Submit your experiment
remote_run = experiment.submit(config=automl_config, show_output=True)

Running on remote.
No run_configuration provided, running on cpu-cluster with default configuration
Running on remote compute: cpu-cluster
Parent Run ID: AutoML_840c15d7-cc55-44fd-8bc0-4507609ca357

Current status: FeaturesGeneration. Generating features for the dataset.
Current status: ModelSelection. Beginning model selection.

****************************************************************************************************
DATA GUARDRAILS: 

TYPE:         Cross validation
STATUS:       DONE
DESCRIPTION:  Each iteration of the trained model was validated through cross-validation.
              
DETAILS:      
+---------------------------------+
|Number of folds                  |
+=================================+
|10                               |
+---------------------------------+

****************************************************************************************************

TYPE:         Class balancing detection
STATUS:       PASSED
DESCRIPTION:  Your inputs were ana

## Run Details

OPTIONAL: Write about the different models trained and their performance. Why do you think some models did better than others?

TODO: In the cell below, use the `RunDetails` widget to show the different experiments.

In [ ]:
RunDetails(remote_run).show()

## Best Model

TODO: In the cell below, get the best model from the automl experiments and display all the properties of the model.



In [ ]:
best_automl_run,fitted_model = remote_run.get_output()
print(best_automl_run)
print(fitted_model)

In [ ]:
best_automl_run

In [ ]:
best_automl_run.get_metrics()

In [ ]:
#TODO: Save the best model
best_automl_run,onnx_model =remote_run.get_output(return_onnx_model=True)
OnnxConverter.save_onnx_model(onnx_model,file_path="./automl_model.onnx")

## Model Deployment

Remember you have to deploy only one of the two models you trained.. Perform the steps in the rest of this notebook only if you wish to deploy this model.

TODO: In the cell below, register the model, create an inference config and deploy the model as a web service.

In [ ]:
#Register the best automl model,
automl_model =remote_run.register_model(model_name = 'automl_best_model')

In [ ]:
script_file_name ='inference/sscore.py'
best_automl_run.download_file('outputs/scoring_file_v_1_0_0.py','inference/sscore.py')

In [ ]:
inference_config = InferenceConfig(entry_script=script_file_name, environment=best_automl_run.get_environment())

In [ ]:
#Local Deployment
from azureml.core.webservice import LocalWebservice
local_config = LocalWebservice.deploy_configuration(port=9001)
local_service = Model.deploy(ws, "test", [automl_model], inference_config, local_config)
local_service.wait_for_deployment(show_output=True)

#Set up the deployment_config as webservice
aci_config = AciWebservice.deploy_configuration(cpu_cores=1, memory_gb=1, enable_app_insights=True)

#Deploy the model
service = Model.deploy(
    workspace = ws,
    name = "heart-failure-model",
    models = [automl_model],
    inference_config = inference_config,
    deployment_config = aci_config, overwrite=True)

#wait until deployment is complete
service.wait_for_deployment(show_output = True)

In [ ]:
print("Scoring uri:" , service.scoring_uri)

TODO: In the cell below, send a request to the web service you deployed to test it.

In [ ]:
pd.DataFrame(test_data).to_csv("data/test_ds.csv", index=False)
test_ds=pd.read_csv("data/test_ds.csv")
#test_ds.drop(test_ds.columns[0],axis=1,inplace=True)
test_ds.drop(test_ds.columns[12],axis=1,inplace=True)
test_ds.info()

In [ ]:
data= test_ds[0:3].to_json(orient="table", index=False)
test_sample=json.dumps(json.loads(data) ,indent=4)
output=service.run(test_sample)
print(output)

In [ ]:
scoring_uri = 'http://f1db25ef-d00c-4743-9ca4-02c25f14a67f.southcentralus.azurecontainer.io/score'
headers = {'Content-Type':'application/json'}

test_data = json.dumps({'data':[{
    'age':75,
    'anaemia':0,
    'creatinine_phosphokinase':582,
    'diabetes':0,
    'ejection_fraction':20,
    'high_blood_pressure':1,
    'platelets':265000,
    'serum_creatinine':1.9,
    'serum_sodium':130,
    'sex':1,
    'smoking':0,
    'time':4}
    ]
        })

test_data2 = json.dumps({'data':[{
    'age':40,
    'anaemia':0,
    'creatinine_phosphokinase':321,
    'diabetes':0,
    'ejection_fraction':35,
    'high_blood_pressure':0,
    'platelets':265000,
    'serum_creatinine':1,
    'serum_sodium':130,
    'sex':1,
    'smoking':0,
    'time':198}
    ]
        })


response = requests.post(scoring_uri, data=test_data2, headers=headers)

print("Result:",response.text)

TODO: In the cell below, print the logs of the web service and delete the service

In [ ]:
#Print the webservice logs
print(service.get_logs())

In [ ]:
#Delete the service
service.delete()